In [2]:
import os
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask, request, jsonify
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline
import glob
from collections import Counter
import logging

In [3]:
# Tải các tài nguyên NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Khởi tạo Flask
app = Flask(__name__)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GIGABYTE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GIGABYTE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from spellchecker import SpellChecker
# Thiết lập logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Khởi tạo spell checker
spell = SpellChecker()

# Tiền xử lý

# PDF

In [5]:
import fitz
import os

In [6]:
# Đường dẫn tới thư mục chứa các file PDF
pdf_folder = r'D:\NLP\medicalbot\dataset\pdf'
output_txt = r'D:\NLP\medicalbot\dataset\pdf_data.txt'

with open(output_txt, 'w', encoding='utf-8') as out_file:
    for filename in os.listdir(pdf_folder):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder, filename)
            doc = fitz.open(pdf_path)
            
            out_file.write(f"\n--- Nội dung từ file: {filename} ---\n\n")
            
            for page in doc:
                text = page.get_text()
                out_file.write(text + '\n')
            
            doc.close()

print("✅ Đã đọc xong toàn bộ file PDF và ghi vào file TXT.")


✅ Đã đọc xong toàn bộ file PDF và ghi vào file TXT.


In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Hàm làm sạch văn bản (giả định, bạn có thể tùy chỉnh)
def clean_text(text):
    import re
    text = re.sub(r'\s+', ' ', text)  # Thay nhiều khoảng trắng bằng một
    text = re.sub(r'[^\w\s.,!?]', '', text)  # Xóa ký tự đặc biệt
    return text.strip()

# Hàm chia đoạn văn bản từ tệp TXT sử dụng RecursiveCharacterTextSplitter
def clean_and_split_txt(txt_path, chunk_size=1000, chunk_overlap=200):
    try:
        # Đọc nội dung tệp TXT
        with open(txt_path, 'r', encoding='utf-8') as f:
            raw_text = f.read()

        # Làm sạch văn bản
        cleaned_text = clean_text(raw_text)

        # Khởi tạo RecursiveCharacterTextSplitter
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            separators=["\n\n", "\n", ". ", " ", ""],  # Thứ tự ưu tiên chia đoạn
        )

        # Chia văn bản thành các đoạn
        chunks = text_splitter.split_text(cleaned_text)

        return chunks
    except Exception as e:
        logging.error(f"Error processing {txt_path}: {str(e)}")
        return []



In [10]:
# Ví dụ sử dụng
if __name__ == "__main__":
    txt_path = r"D:\NLP\medicalbot\dataset\pdf_data.txt"
    chunks = clean_and_split_txt(txt_path)
    logging.info(f"Extracted {len(chunks)} chunks")
    # (Tùy chọn) Lưu các đoạn vào tệp hoặc xử lý tiếp
    with open(txt_path.replace(".txt", "_chunks.txt"), 'w', encoding='utf-8') as f:
        for i, chunk in enumerate(chunks):
            f.write(f"Chunk {i+1}:\n{chunk}\n\n")

2025-05-08 17:32:55,564 - INFO - Extracted 57737 chunks


# Q&A

In [20]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'\s+', ' ', text.strip())  # Loại bỏ khoảng trắng thừa
    text = re.sub(r'[^\w\s.,!?]', '', text)  # Loại bỏ ký tự đặc biệt
    text = text.lower()  # Chuẩn hóa chữ thường
    # Sửa lỗi chính tả
    words = text.split()
    corrected_words = [spell.correction(word) if spell.correction(word) else word for word in words]
    text = ' '.join(corrected_words)
    # Sửa lỗi cụ thể từ bộ dữ liệu
    corrections = {
        'ejackulated': 'ejaculated',
        'turk': 'turkey',
        'cronic': 'chronic',
        'bhp': 'benign prostatic hyperplasia'
    }
    for wrong, correct in corrections.items():
        text = text.replace(wrong, correct)
    return text

# Hàm kiểm tra chất lượng Q&A
def check_qa_quality(df):
    report = {}
    
    # Kiểm tra nhãn
    labels = df['label'].value_counts()
    report['label_distribution'] = labels.to_dict()
    
    # Kiểm tra độ dài câu hỏi và trả lời
    df['question_length'] = df['short_question'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
    df['answer_length'] = df['short_answer'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
    report['question_length_stats'] = {
        'mean': df['question_length'].mean(),
        'min': df['question_length'].min(),
        'max': df['question_length'].max()
    }
    report['answer_length_stats'] = {
        'mean': df['answer_length'].mean(),
        'min': df['answer_length'].min(),
        'max': df['answer_length'].max()
    }
    
    # Kiểm tra giá trị thiếu
    report['missing_values'] = df.isnull().sum().to_dict()
    
    # Phân tích tags
    all_tags = []
    for tags in df['tags'].dropna():
        tags_list = eval(tags) if tags.startswith('[') else tags.split(',')
        all_tags.extend([tag.strip().lower() for tag in tags_list])
    report['top_tags'] = Counter(all_tags).most_common(10)
    
    return report

In [19]:
def process_qa_dataset(input_csv, output_csv):
    logging.info("Processing Q&A dataset...")
    
    # Đọc dữ liệu
    try:
        df = pd.read_csv(input_csv)
    except Exception as e:
        logging.error(f"Error reading {input_csv}: {str(e)}")
        return None, None
    
    # Kiểm tra nhãn
    if not all(df['label'] == 1.0):
        logging.warning("Found labels other than 1.0, filtering to keep only label 1.0")
        df = df[df['label'] == 1.0]
    
    # Làm sạch dữ liệu
    df['short_question'] = df['short_question'].apply(clean_text)
    df['short_answer'] = df['short_answer'].apply(clean_text)
    df['tags'] = df['tags'].fillna('[]')  # Thay giá trị thiếu trong tags bằng danh sách rỗng
    
    # Loại bỏ các mẫu không hợp lệ
    df = df[df['short_question'].str.len() > 5]  # Loại câu hỏi quá ngắn
    df = df[df['short_answer'].str.len() > 10]   # Loại câu trả lời quá ngắn
    
    # Chuyển đổi định dạng cho tinh chỉnh
    df['text'] = df.apply(lambda row: f"Question: {row['short_question']}\nTags: {row['tags']}\nAnswer: {row['short_answer']}", axis=1)
    
    # Kiểm tra chất lượng
    quality_report = check_qa_quality(df)
    
    # Lưu dữ liệu đã xử lý
    df[['text', 'short_question', 'short_answer', 'tags', 'label']].to_csv(output_csv, index=False)
    logging.info(f"Processed Q&A dataset saved to {output_csv}")
    
    return df, quality_report


In [ ]:
# Đường dẫn dữ liệu
qa_input = r"D:\NLP\medicalbot\dataset\qa_base\validation_data_chatbot.csv"
qa_output = r"D:\NLP\medicalbot\dataset\qa_clear\clear_val_data1111.csv"

# Xử lý Q&A
qa_df, qa_report = process_qa_dataset(qa_input, qa_output)
if qa_df is not None:
    print("Q&A Processing Report:")
    print(f"Number of samples: {len(qa_df)}")
    print(f"Label distribution: {qa_report['label_distribution']}")
    print(f"Question length stats: {qa_report['question_length_stats']}")
    print(f"Answer length stats: {qa_report['answer_length_stats']}")
    print(f"Missing values: {qa_report['missing_values']}")
    print(f"Top 10 tags: {qa_report['top_tags']}")

In [26]:
# Đọc file CSV
df = pd.read_csv(r'D:\NLP\medicalbot\dataset\qa_clear\clear_val_data.csv')

def clean_text(text):
    # Tìm nội dung trong cặp dấu ngoặc
    match = re.search(r'\[(.*?)\]', text)
    if match:
        # Lấy nội dung trong dấu ngoặc
        content = match.group(1)
        # Xóa dấu nháy và chia thành danh sách, sau đó nối lại bằng ", "
        cleaned_content = ', '.join(item.strip("' ") for item in content.split("' '"))
        # Thay thế cặp dấu ngoặc và nội dung bằng nội dung đã xử lý
        cleaned_text = re.sub(r'\[.*?\]', cleaned_content, text)
    else:
        # Nếu không có cặp dấu ngoặc, giữ nguyên text
        cleaned_text = text
    
    return cleaned_text

# Áp dụng hàm clean_text vào cột 'text'
df['text'] = df['text'].apply(clean_text)

# Lưu lại file CSV đã chỉnh sửa
df.to_csv(r'D:\NLP\medicalbot\dataset\qa_clear\clear_val_data.csv', index=False)

print("Đã xử lý cột 'text' và lưu vào file 'clear_val_data_cleaned.csv'")

Đã xử lý cột 'text' và lưu vào file 'clear_val_data_cleaned.csv'


# Phần Chatbot Q&A

In [6]:
import pandas as pd
from datasets import Dataset

# Đọc file dữ liệu
train_df = pd.read_csv(r"D:\NLP\medicalbot\dataset\qa_clear\clear_train_data.csv")
val_df = pd.read_csv(r"D:\NLP\medicalbot\dataset\qa_clear\clear_val_data.csv")

# Chuyển thành định dạng Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df[['text']])
val_dataset = Dataset.from_pandas(val_df[['text']])

# In thông tin
print(f"Tập huấn luyện: {len(train_dataset)} mẫu")
print(f"Tập kiểm tra: {len(val_dataset)} mẫu")
print("Ví dụ prompt đầu tiên:")
print(train_dataset[0]['text'])

2025-05-11 09:44:38,712 - INFO - PyTorch version 2.5.1+cu121 available.
2025-05-11 09:44:38,712 - INFO - TensorFlow version 2.19.0 available.


Tập huấn luyện: 23663 mẫu
Tập kiểm tra: 5915 mẫu
Ví dụ prompt đầu tiên:
Question: can an antibiotic through an i give you a rash a couple days later
Tags: rash, antibiotic
Answer: yes it can even after you have finished the prescription for antibiotics


In [7]:
import torch
print("Is CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


Is CUDA available: True
GPU name: NVIDIA GeForce RTX 3050 Laptop GPU


In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import numpy
import os

# Giải phóng VRAM
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()  # Giải phóng VRAM trước khi huấn luyện

# Thêm numpy.dtypes.UInt32DType, numpy.dtype, numpy.ndarray và numpy._core.multiarray._reconstruct vào danh sách an toàn
torch.serialization.add_safe_globals([
    numpy.dtypes.UInt32DType,
    numpy.dtype,
    numpy.ndarray,
    numpy._core.multiarray._reconstruct
])

# Kiểm tra GPU và VRAM
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"Available VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Tải mô hình và tokenizer
model_name = "meta-llama/Llama-3.2-1B-Instruct"  # Thay bằng đường dẫn thực tế
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Sử dụng float16 để tiết kiệm bộ nhớ
    device_map="auto",         # Tự động phân bổ lên GPU
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Đặt pad_token

# Tải dataset từ file CSV
dataset = load_dataset("csv", data_files={"train": r"D:\NLP\medicalbot\dataset\qa_clear\clear_train_data.csv", "validation": r"D:\NLP\medicalbot\dataset\qa_clear\clear_val_data.csv"})
train_dataset = dataset["train"]
val_dataset = dataset["validation"]

# Hàm tokenize cho dataset
def tokenize_function(examples):
    texts = examples["text"]
    tokenized = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=512,  # Có thể giảm xuống 256 nếu cần tiết kiệm VRAM
        return_tensors="pt"
    )
    tokenized["labels"] = tokenized["input_ids"].clone()
    tokenized["labels"][tokenized["labels"] == tokenizer.pad_token_id] = -100
    return tokenized

# Áp dụng tokenize cho dataset
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=val_dataset.column_names)

# Định dạng dataset để huấn luyện
train_dataset.set_format("torch")
val_dataset.set_format("torch")

# Cấu hình LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# Cấu hình huấn luyện
training_args = TrainingArguments(
    output_dir="./lora_finetuned_llama",
    eval_strategy="epoch",
    per_device_train_batch_size=4,  # Theo yêu cầu
    per_device_eval_batch_size=4,   # Theo yêu cầu
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,                     # Bật mixed precision để tối ưu GPU
    save_strategy="epoch",         # Lưu mô hình sau mỗi epoch
    load_best_model_at_end=True,
    dataloader_pin_memory=True,    # Tăng tốc truyền dữ liệu sang GPU
    dataloader_num_workers=4,      # Sử dụng nhiều worker để tải dữ liệu
    logging_steps=10,              # Ghi log thường xuyên để theo dõi
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=None  # Không cần collator vì đã padding trong tokenize
)

# Tiếp tục huấn luyện từ checkpoint
checkpoint_path = "D:/NLP/medicalbot/lora_finetuned_llama/checkpoint-5916"
trainer.train(resume_from_checkpoint=checkpoint_path)

# Lưu mô hình và tokenizer
model.save_pretrained("./lora_finetuned_llama")
tokenizer.save_pretrained("./lora_finetuned_llama")

# Giải phóng VRAM sau huấn luyện
torch.cuda.empty_cache()

Using device: cuda
Available VRAM: 4.29 GB


c:\Users\GIGABYTE\anaconda3\envs\env_medbot\lib\site-packages\accelerate\utils\modeling.py:1569: UserWarning: Current model requires 64 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
2025-05-11 09:45:43,444 - INFO - Based on the current allocation process, no modules could be assigned to the following devices due to insufficient memory:
  - 0: 646979584 bytes required
These minimum requirements are specific to this allocation attempt and may vary. Consider increasing the available memory for these devices to at least the specified minimum, or adjusting the model config.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
c:\Users\GIGABYTE\anaco

OutOfMemoryError: CUDA out of memory. Tried to allocate 502.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 10.53 GiB is allocated by PyTorch, and 42.08 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Phần RAG

In [ ]:
import pdfplumber

with pdfplumber.open(r"D:\NLP\medicalbot\Gale Encyclopedia of Medicine. Vol. 1. 2nd ed.pdf") as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    with open("output.txt", "w", encoding="utf-8") as txt_file:
        txt_file.write(text)

In [ ]:
loader = TextLoader('output.txt')
documents = loader.load()

# Chia nhỏ tài liệu
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# Tạo vector store với Chroma và embedding
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(texts, embeddings)

# Khởi tạo mô hình ngôn ngữ (sử dụng pipeline nhỏ để demo)
llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",  # Thay bằng mô hình mạnh hơn như Llama nếu có GPU
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 150}
)

# Tạo chuỗi RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

# Hàm lấy câu trả lời từ RAG
def get_rag_answer(user_question):
    result = qa_chain({"query": user_question})
    answer = result['result']
    return f"Based on the Gale Encyclopedia of Medicine: {answer}\n\nPlease consult a healthcare professional for personalized advice."


Device set to use cpu


# Kết hợp cả hai

In [13]:
def get_hybrid_answer(user_question):
    # Thử tìm câu trả lời từ dữ liệu hội thoại trước
    conv_answer, conv_score = get_conversation_answer(user_question)
    
    if conv_answer and conv_score > 0.5:  # Nếu câu trả lời hội thoại đủ tốt
        return f"Here's a quick answer: {conv_answer}\n\nFor more details, consult a healthcare professional."
    else:
        # Sử dụng RAG để lấy thông tin chi tiết
        rag_answer = get_rag_answer(user_question)
        if conv_answer:
            return f"Here's a quick answer: {conv_answer}\n\nFor more details: {rag_answer}"
        return rag_answer


In [ ]:
# API endpoint cho chatbot
@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get('message', '')
    response = get_hybrid_answer(user_input)
    return jsonify({'response': response})

if __name__ == '__main__':
    app.run(debug=True)